# FER-2013 Dataset


### **FER-2013 avec ResNet18**

**📚 2. Importation des bibliothèques**

In [2]:
import torch

print("CUDA disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun")


CUDA disponible : False
Nom du GPU : Aucun


In [1]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())


None
False


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print("🖥️  Utilisation du périphérique :", device)

False
🖥️  Utilisation du périphérique : cpu


**📥 3. Définition du dataset FER-2013**

In [ ]:
class FER2013Dataset(Dataset):
    def __init__(self, csv_path, usage="Training", transform=None):
        self.data = pd.read_csv(csv_path)
        self.data = self.data[self.data["Usage"] == usage]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        emotion = int(self.data.iloc[idx]["emotion"])
        img = np.fromstring(self.data.iloc[idx]["pixels"], sep=" ", dtype=np.uint8).reshape(48, 48)
        img = Image.fromarray(img).convert("L")
        if self.transform:
            img = self.transform(img)
        return img, emotion

**🧰 4. Prétraitement des images**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Adapter à ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

**📂 5. Chargement des données**

In [ ]:
train_dataset = ImageFolder(root='../data/FER-2013/train', transform=transform)
val_dataset = ImageFolder(root='../data/FER-2013/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

**🧠 6. Création du modèle ResNet18**

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)  # 7 classes d’émotions

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

**⚙️ 7. Fonction d'entraînement et d'évaluation**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return running_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return val_loss / len(loader), acc

**🚀 8. Boucle d'entraînement**

In [ ]:
num_epochs = 1
train_acc_list, val_acc_list = [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    print(f"Époque {epoch+1} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


**📈 9. Courbe de performance**

In [ ]:
plt.plot(train_acc_list, label="Train Accuracy")
plt.plot(val_acc_list, label="Validation Accuracy")
plt.xlabel("Époque")
plt.ylabel("Précision")
plt.title("Performance du modèle")
plt.legend()
plt.show()